In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter1.util.importer import *
MODEL_PATH = 'model_saves/ex4_model'
uk_path = "../../src/data_preprocess/dataset/UKDATA_cleaned.csv"

In [ ]:
assert time_horizon > 0, "Time horizon must be a positive integer"
    
df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = GRU(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .build()

model.fit()
torch.save(model.state_dict(), MODEL_PATH + ".pth")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Finding best initial lr: 100%|██████████| 100/100 [00:00<00:00, 119.78it/s]
Learning rate set to 0.003981071705534969
Restoring states from the checkpoint path at /home/vind/P5/experimentation/experiments/iter1/.lr_find_f4af41b1-fca2-4fe4-9c9d-551d6bc2c356.ckpt
Restored all states from the checkpoint at /home/vind/P5/experimentation/experiments/iter1/.lr_find_f4af41b1-fca2-4fe4-9c9d-551d6bc2c356.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 2 succeeded, trying batch size 4
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 4 succeeded, trying batch size 8
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 8 succeeded, trying batch size 16
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 16 succeeded, trying batch size 32
`Trainer.fit` stopped: `max_steps

Epoch 405:  62%|██████▏   | 176/284 [00:01<00:00, 113.50it/s, v_num=331, train_loss_step=0.0143, val_loss=1.480, train_loss_epoch=0.0213]   

In [ ]:
from experiments.iter1.util.evaluate import evaluate_model

df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = GRU(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .build()

model.load_state_dict(torch.load(MODEL_PATH + ".pth", weights_only=True))
model.test()

evaluate_model(model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, seq_len, time_horizon, TARGET_COLUMN, error, temp_boundary, None)